In [1]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind, f_oneway
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_squared_error

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
#list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
#neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
#reviews = pd.read_csv('inside_airbnb/reviews.csv')
#reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
listings.columns

Index([u'id', u'listing_url', u'scrape_id', u'last_scraped', u'name',
       u'summary', u'space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules', u'thumbnail_url', u'medium_url',
       u'picture_url', u'xl_picture_url', u'host_id', u'host_url',
       u'host_name', u'host_since', u'host_location', u'host_about',
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_thumbnail_url', u'host_picture_url',
       u'host_neighbourhood', u'host_listings_count',
       u'host_total_listings_count', u'host_verifications',
       u'host_has_profile_pic', u'host_identity_verified', u'street',
       u'neighbourhood', u'neighbourhood_cleansed',
       u'neighbourhood_group_cleansed', u'city', u'state', u'zipcode',
       u'market', u'smart_location', u'country_code', u'country', u'latitude',
       u'longitude', u'is_location_exact', u'prope

In [5]:
listings['transit'].head()

0                                                  NaN
1    Getting around is easy. Very close to Eastlake...
2                                                  NaN
3                                                  NaN
4    Sorry, no buses within 1.5 mile; plenty of str...
Name: transit, dtype: object

In [6]:
listings['transit'][6606]

'For better or worse, San Diego is a driving city.  Everyone drives and cars are pretty much the best way to get around.   That being said, the apartment is very close to a bus stop, which will take you around Solana Beach.  The MTS is a great bus service that can take you downtown, or all over San Diego.  We also have a great trolly system.   Taxis, Uber and Lyft are also really good ways to commute in San Diego, especially if you go to bars.   Rental cars and Car2go are great ways to get around as well.'

In [7]:
listings['transit'][4]

'Sorry, no buses within 1.5 mile; plenty of street parking if you have a car.  Soon to be a "Rapid" bus route from Otay Mesa border crossing, through our neighborhood, and on to downtown San Diego (dedicated stops only).  Uber is popular here.'

In [8]:
listings['smart_location'].head()

0      San Diego, CA
1    Chula Vista, CA
2    Chula Vista, CA
3    Chula Vista, CA
4    Chula Vista, CA
Name: smart_location, dtype: object

In [9]:
listings['cancellation_policy'].unique()

array(['moderate', 'strict', 'flexible', 'super_strict_60',
       'super_strict_30', 'no_refunds'], dtype=object)

In [10]:
def comma_split_sum(x):
    b = x.split(',')
    v = ''
    
    for h in b:
        v = v + h
    
    return v

In [11]:
listings['numerical_price'] = listings['price'].map(lambda x: x[1:]).map(comma_split_sum).map(lambda x: float(x))

In [12]:
for w in listings['cancellation_policy'].unique():
    a = listings[listings['cancellation_policy'] == w]['numerical_price'].mean()
    print(w, a)

('moderate', 140.21585609593603)
('strict', 230.4646814404432)
('flexible', 175.4077568134172)
('super_strict_60', 687.2217573221757)
('super_strict_30', 370.3380281690141)
('no_refunds', 295.0)


In [13]:
L = []
for w in listings['cancellation_policy'].unique():
    a = listings[listings['cancellation_policy'] == w]['numerical_price']
    L.append(a)

In [14]:
len(L)

6

In [15]:
f_oneway(L[0], L[1], L[2], L[3], L[4], L[5])

F_onewayResult(statistic=208.31891371991685, pvalue=6.9310916152157314e-207)

In [16]:
listings['instant_bookable'].head()

0    f
1    f
2    f
3    f
4    f
Name: instant_bookable, dtype: object

In [17]:
K = []
for w in listings['instant_bookable'].unique():
    a = listings[listings['instant_bookable'] == w]['numerical_price']
    K.append(a)

In [18]:
len(K)

2

In [19]:
ttest_ind(K[0], K[1])

Ttest_indResult(statistic=-6.258843617575387, pvalue=4.1214250346295871e-10)

In [20]:
listings['property_type'].head()

0          House
1    Condominium
2    Condominium
3      Townhouse
4          House
Name: property_type, dtype: object

In [21]:
H = []
for w in listings['property_type'].unique():
    a = listings[listings['property_type'] == w]['numerical_price']
    H.append(a)

In [22]:
len(H)

17

In [23]:
listings['numerical_cleaning_fee'] = listings['cleaning_fee'].fillna('$0.00').map(lambda x: x[1:]).map(comma_split_sum).map(lambda x: float(x))

# Modeling 

In [24]:
linear_regression = linear_model.LinearRegression(normalize=False, fit_intercept=True)

In [25]:
listings['numerical_cleaning_fee'].shape, listings['numerical_price'].shape

((6608L,), (6608L,))

In [26]:
np.corrcoef(listings['numerical_cleaning_fee'], listings['bedrooms'])

array([[  1.,  nan],
       [ nan,  nan]])

In [27]:
len(listings['bedrooms'].dropna())

6605

In [28]:
features = ['numerical_cleaning_fee', 'bedrooms']
X = listings[features].dropna()
np.corrcoef(X['numerical_cleaning_fee'], X['bedrooms'])

array([[ 1.        ,  0.66934584],
       [ 0.66934584,  1.        ]])

### Linear Regression

In [29]:
features = ['numerical_cleaning_fee', 'numerical_price']
data = listings[features]
model = sm.ols(formula = 'numerical_price ~ numerical_cleaning_fee', data = data).fit()

In [30]:
model.params

Intercept                 80.971109
numerical_cleaning_fee     1.717117
dtype: float64

In [31]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        numerical_price   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.376
Method:                 Least Squares   F-statistic:                     3988.
Date:                Wed, 18 Apr 2018   Prob (F-statistic):               0.00
Time:                        13:24:20   Log-Likelihood:                -44880.
No. Observations:                6608   AIC:                         8.976e+04
Df Residuals:                    6606   BIC:                         8.978e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 80.9711      3.367     24.046      0.000      74.370      87.572
numerical_cleaning_fee     1.7171      0.027     63.150      0.000       1.664       1.770
==============================================================================
Omnibus:                     8385.640   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2157363.382
Skew:                           6.811   Prob(JB):                         0.00
Kurtosis:                      90.464   Cond. No.                         157.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Quadratic Regression

In [32]:
features = ['numerical_cleaning_fee', 'numerical_price']
data = listings[features]
model = sm.ols(formula = 'numerical_price ~ np.power(numerical_cleaning_fee, 2) + numerical_cleaning_fee', data = data).fit()

In [33]:
model.params

Intercept                              120.415816
np.power(numerical_cleaning_fee, 2)      0.002804
numerical_cleaning_fee                   0.637534
dtype: float64

In [34]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        numerical_price   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                     2390.
Date:                Wed, 18 Apr 2018   Prob (F-statistic):               0.00
Time:                        13:24:22   Log-Likelihood:                -44641.
No. Observations:                6608   AIC:                         8.929e+04
Df Residuals:                    6605   BIC:                         8.931e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                             120.4158      3.701     32.537      0.000     113.161     127.671
np.power(numerical_cleaning_fee, 2)     0.0028      0.000     22.238      0.000       0.003       0.003
numerical_cleaning_fee                  0.6375      0.055     11.554      0.000       0.529       0.746
==============================================================================
Omnibus:                     8321.296   Durbin-Watson:                   1.943
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2419785.859
Skew:                           6.653   Prob(JB):                         0.00
Kurtosis:                      95.798   Cond. No.                     6.56e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.56e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Cubic Regression

In [35]:
features = ['numerical_cleaning_fee', 'numerical_price']
data = listings[features]
model = sm.ols(formula = 'numerical_price ~ np.power(numerical_cleaning_fee, 3) + np.power(numerical_cleaning_fee, 2) + numerical_cleaning_fee', data = data).fit()

In [36]:
model.params

Intercept                              133.662676
np.power(numerical_cleaning_fee, 3)     -0.000004
np.power(numerical_cleaning_fee, 2)      0.006194
numerical_cleaning_fee                   0.033082
dtype: float64

In [37]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        numerical_price   R-squared:                       0.425
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     1629.
Date:                Wed, 18 Apr 2018   Prob (F-statistic):               0.00
Time:                        13:24:25   Log-Likelihood:                -44610.
No. Observations:                6608   AIC:                         8.923e+04
Df Residuals:                    6604   BIC:                         8.925e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                             133.6627      4.044     33.050      0.000     125.735     141.591
np.power(numerical_cleaning_fee, 3) -3.949e-06   4.98e-07     -7.935      0.000   -4.92e-06   -2.97e-06
np.power(numerical_cleaning_fee, 2)     0.0062      0.000     13.910      0.000       0.005       0.007
numerical_cleaning_fee                  0.0331      0.094      0.352      0.725      -0.151       0.217
==============================================================================
Omnibus:                     8281.653   Durbin-Watson:                   1.937
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2355401.607
Skew:                           6.601   Prob(JB):                         0.00
Kurtosis:                      94.545   Cond. No.                     4.06e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.06e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### All polynomial regression

In [39]:
for d in range(1,4):
    create_poly = PolynomialFeatures(degree=d, interaction_only=False, include_bias=False)
    poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
    model = poly.fit(data, listings['numerical_price'])
    print ("R2 degree - %2i polynomial : %0.3f" %(d,r2_score(listings['numerical_price'], model.predict(data))))

R2 degree -  1 polynomial : 1.000
R2 degree -  2 polynomial : 1.000
R2 degree -  3 polynomial : 1.000


In [40]:
listings['numerical_cleaning_fee'].shape

(6608L,)

In [41]:
model.predict(listings['numerical_cleaning_fee']).shape

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: X shape does not match training shape

In [48]:
r2_score(listings['numerical_price'], model.predict(listings['numerical_cleaning_fee']))

0.42535299239996349

In [62]:
type(listings['numerical_cleaning_fee'].values)

numpy.ndarray

In [68]:
create_poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(data, listings['numerical_price'])
r2_score(listings['numerical_price'], model.predict(data))

1.0

In [ ]:
create_poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(data, listings['numerical_price'])
r2_score(listings['numerical_price'], model.predict(data))